In [1]:
import numpy as np
from scipy.stats import gamma

class IDISO:
    def __init__(self, NWU, MIT, LB, UB, DIM, OBJ):
        self.NWU = NWU
        self.MIT = MIT
        self.LB = LB
        self.UB = UB
        self.DIM = DIM
        self.OBJ = OBJ

    def Cauchy_PDF(self, x, m, b):
        return (1 / np.pi) * (b / ((x - m)**2 + b**2))

    def run(self):
        NWU, MIT, LB, UB, DIM, OBJ = self.NWU, self.MIT, np.array(self.LB), np.array(self.UB), self.DIM, self.OBJ

        
        curve = np.zeros((MIT, 1))
        Fit_V = np.zeros((NWU, 1))
        Water_X = np.random.uniform(LB, UB, (NWU, DIM))
        Pbest = Fit_V.copy()
        V = np.zeros((NWU, DIM)) 
        P_Pobest = Water_X.copy()
        Best_pos = np.zeros(DIM)
        Fit_Best = np.inf
        PositionValue = [None] * MIT

        MLV = np.zeros(NWU)
        LP = np.zeros(NWU)
        DI = np.zeros(NWU)
        RCF = np.zeros(NWU)

     
        SDR, WDR1, WDR2 = 0.3, 0.23, 0.68
        CFR, CRC = 9.435, -1.5
        WDI, VDC = 1.35, 0.46
        HGO, HGI = 1.3, 0.82
        HRO, HRI = 1.2, 7.2
        GMP, GMQ = 0.85, 2.5
        CF = np.zeros(MIT)
        
      
        
        for j in range(MIT):
       
            CF[j] = CFR * gamma.pdf(j / MIT, GMP, scale=GMQ)
            
            for i in range(NWU):
                R1 = np.random.rand()
                R2 = np.random.rand(DIM)
                R3 = np.random.rand(DIM)
                if R1 < WDR1:
                    V[i, :] = R2 * HRO**(2/3) * HGO**(1/2) / CF[j]
                else:
                    V[i, :] = R3 * HRI**(2/3) * HGI**(1/2) / CF[j]

                KK = (-1)**np.random.randint(0, 2) * (Water_X[i, :] - np.mean(Water_X, axis=0)) * np.random.rand(DIM)
                Cmax = np.exp(3)
                Cmin = np.exp(2)
                C = np.log((Cmax - Cmin) * (MIT - j) / (MIT + np.finfo(float).eps) + Cmin)
                Water_X[i, :] = Best_pos + (Best_pos - Water_X[i, :]) * np.random.rand(DIM) * V[i, :] + KK * C * gamma.pdf(j, GMP, scale=GMQ)

                
                Flag_UB = Water_X[i, :] > UB
                Flag_LB = Water_X[i, :] < LB
                Water_X[i, :] = Water_X[i, :] * (~(Flag_UB | Flag_LB)) + UB * Flag_UB + LB * Flag_LB

                
                Fit_V[i, 0] = OBJ(Water_X[i, :])
                if Fit_V[i, 0] < Pbest[i, 0]:
                    Pbest[i, 0] = Fit_V[i, 0]
                    P_Pobest[i, :] = Water_X[i, :]

                if Fit_V[i, 0] < Fit_Best:
                    Fit_Best = Fit_V[i, 0]
                    Best_pos = Water_X[i, :]
                if np.isnan(Water_X[i, :]).any():
                    Water_X[i, :] = np.random.uniform(LB, UB, DIM)
            for i in range(NWU):
                R1 = np.random.rand()
                R4 = np.random.rand(DIM)
                R5 = np.random.rand(DIM)
                R6 = np.random.rand(DIM)
                R7 = np.random.rand(DIM)
                MLV[i] = np.mean(Fit_V)
                LP[i] = WDI * VDC * MLV[i]**2 / CRC  # LP = lateral pressure 计算侧压力

                DI[i] = WDI * np.cos(np.radians(90 * j / MIT))
                RCF[i] = DI[i] * np.sqrt(np.sum((Best_pos - Water_X[i, :])**2)) / CRC  # 计算离心力
                if RCF[i] - LP[i] > 0:
                    MFI = np.argmax(Fit_V[i:NWU, :])
                    R6 = np.random.rand(DIM)
                    Water_X[MFI, :] = R6 * (UB - LB) + LB
                if R1 < WDR2:  # WDR2为第二次内外分水比例
                    V[i, :] = R4 * 0.5 * HRI**(2/3) * HGI**(1/2) / CF[j]  # 计算内江离堆左侧流速
                else:
                    V[i, :] = R5 * HRI**(2/3) * HGI**(1/2) / CF[j]  
                index = np.argsort(np.abs(Fit_V - Fit_V[i, 0]))
                Q = np.exp(-(1 - j / MIT) * 2)
                Water_X[i, :] = Water_X[i, :] + (P_Pobest[i, :] - Water_X[i, :]) * V[i, :] * np.random.rand(DIM) * 2 * Q + \
                            np.sign(Fit_V[index[1], 0] - Fit_V[i, 0]) * (Water_X[index[1], :] - Water_X[i, :]) * np.random.rand(DIM)

                
                Water_X1 = Best_pos +self.Cauchy_PDF(j, 0, 1) * Water_X[i, :]
                if OBJ(Water_X1) < OBJ(Water_X[i, :]):
                    Water_X[i, :] = Water_X1

                Flag_UB = Water_X[i, :] > UB
                Flag_LB = Water_X[i, :] < LB
                Water_X[i, :] = Water_X[i, :] * (~(Flag_UB | Flag_LB)) + UB * Flag_UB + LB * Flag_LB

        
            NFindex = np.argsort(-Fit_V, axis=0).ravel()
            Temp_X = Water_X[NFindex, :]
            nw = int(np.ceil(NWU * SDR))
            DesiltingWater_X = np.random.rand(nw, DIM) * Temp_X[:nw, :]
            RemWater_X = Temp_X[nw:, :]
            New_X = np.vstack([DesiltingWater_X, RemWater_X])
            b = np.random.permutation(NWU)
            Water_X = New_X[b, :]

         
            curve[j, 0] = Fit_Best
            PositionValue[j] = [Best_pos.copy(), Fit_Best, Water_X.copy(), Fit_V.copy()]

        return Fit_Best, Best_pos, curve, PositionValue




In [51]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import pandas as pd


data = pd.read_csv('../data.csv') 
x = np.array(data.iloc[:,:9])
y = np.array(data.iloc[:,11])
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5,random_state=42)

def xgb_objective(params):
    
    if np.isnan(params).any():
        return 1e9  

    max_depth = int(max(1, np.round(params[0])))
    n_estimators = int(max(1, np.round(params[4])))
    
    alpha = max(params[2], 0)
    
    learning_rate = max(params[3], 0)
    subsample = max(params[1], 0)
    
    model = xgb.XGBRegressor(max_depth=max_depth, learning_rate=learning_rate,
                              alpha = alpha ,subsample = subsample ,n_estimators = n_estimators)
    
    model.fit(X_train, y_train)

    
    predictions = model.predict(X_val)
    accuracy = r2_score(y_val, predictions)


    return -accuracy


In [ ]:


LB = [3, 0.001,0.01,0.01,100]  
UB = [10, 0.3,0.05,1,1000]  


optimizer = IDISO(NWU=30, MIT=20, LB=LB, UB=UB, DIM=5, OBJ=xgb_objective)


best_score, best_params, diso_curve, position_value = optimizer.run()

print("Best Score:", -best_score)  
print("Best Parameters:", best_params)


In [ ]:

model = xgb.XGBRegressor(max_depth=8,
                         subsample = 5.5392384E-1,
                          alpha = 3.5553879E-2 ,
                          learning_rate= 8.1734951E-2,
                              n_estimators = 533)
x = np.array(data.iloc[:,:9])
y = np.array(data.iloc[:,12])
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=2020)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5,random_state=20200)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
val_predictions = model.predict(X_val)
r2_1 = r2_score(y_test, predictions)
r2_2 = r2_score(y_val,val_predictions)
r2_1,r2_2

In [ ]:
import matplotlib.pyplot as plt
import os


plt.figure(figsize=(8, 6))
plt.scatter(y_test, predictions, label='Test Set', alpha=0.7)
plt.scatter(y_val, val_predictions, label='Validation Set', alpha=0.7)

plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('N Actual vs. Predicted Values for Test and Validation Sets')


max_value = max(y_test.max(), y_val.max())
plt.plot([0, max_value], [0, max_value], linestyle='--', color='gray')



plt.legend()

folder_path = '../Desktop/超参数/'
new_file_name = 'N'
save_path = os.path.join(folder_path, new_file_name)
plt.savefig(save_path, dpi=600)
plt.show()

plt.show()
